# Notice
## This python code is an unofficial Tensorflow implementation of the model from following reference
### Authors : Havaei,M.; Davy,A.; Warde-Farley,D.; Biard,A.; Courville,A.; Bengio,Y.; Pal,C.; Jodoin,P.M.; Larochelle,H. 
### Title : Brain tumor segmentation with Deep Neural Networks
### Source : Med.Image Anal., 2017, 35, 18-31, Elsevier B.V, Netherlands
###### The code has not been completed yet. 

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

In [ ]:
import numpy as np
import random
import json
import h5py
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
from skimage import io, color, img_as_float
from skimage.exposure import adjust_gamma
from skimage.segmentation import mark_boundaries
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.metrics import classification_report

In [ ]:
import tensorflow as tf

In [40]:
import SimpleITK as sitk
from ipywidgets import interact, FloatSlider
from myshow import myshow, myshow3d
from TwoPathCNN import vanilla_CNN

In [ ]:
class Cascaded_architecture(vanilla_CNN, object):
    def CNN_for_INPUT_Cascade(self, x):
        cascadedLayer = self.Defining_CNN(x, 65)
        y_conv = self.Defining_CNN(x, 33, ("InputCas", cascadedLayer))
        return y_conv
    
    def CNN_for_LOCAL_Cascade(self, x):
        cascadedLayer = self.Defining_CNN(x, 56)
        y_conv = self.Defining_CNN(x, 33, ("LocalCas", cascadedLayer))
        return y_conv
    
    def CNN_for_MF_Cascade(self, x):
        cascadedLayer = self.Defining_CNN(x, 53)
        y_conv = self.Defining_CNN(x, 33, ("MFCas", cascadedLayer))
        return y_conv
        
        

In [ ]:
def Feeding_Data_and_Realizing_the_Model(_):
    # data를 import한다.
    img_T1 = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_T1.54567/VSD.Brain.XX.O.MR_T1.54567.mha")
    img_T2 = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_T2.54569/VSD.Brain.XX.O.MR_T2.54569.mha")
    img_FLAIR = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_Flair.54566/VSD.Brain.XX.O.MR_Flair.54566.mha")
    img_T1c = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_T1c.54568/VSD.Brain.XX.O.MR_T1c.54568.mha")

    # 모델을 생성한다.
    img_T1_Slice = img_T1[:,:,77]
    img_T1c_Slice = img_T1c[:,:,77]
    img_T2_Slice = img_T2[:,:,77]
    img_FLAIR_Slice = img_FLAIR[:,:,77]
    imgVolume = sitk.JoinSeries(img_T1_Slice, img_T1c_Slice, img_T2_Slice, img_FLAIR_Slice)
    width = imgVolume.GetWidth()
    height = imgVolume.GetHeight()
    depth = imgVolume.GetDepth()
    X = tf.placeholder(tf.float32, shape=[None, width*height*depth])
    ### NOTE: If you call this function often, you may want to (i) move the `np.pad()`
    #### into the graph as `tf.pad()`, and (ii) replace `X_t` with a placeholder.
    ###X = np.pad(X, (pad_amount, 0), 'constant')
    ###X_t  = tf.convert_to_tensor(X)
    ###output_t = tf.map_fn(window_func, tf.range(window_size))
    ###output = sess.run(output_t)

    # loss와 optimizer를 정의한다.
    y_ = tf.placeholder(tf.float32, [None, 10])

    # Deep Neural Networks 그래프를 생성한다.
    y_conv = Defining_Two_pathway_CNN(X)

    # Cross Entropy를 비용함수(loss function)으로 정의하고, AdamOptimizer를 이용해서 비용 함수를 최소화한다.
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
  
    # 정확도를 측정한다.
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # Dice coefficient

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(20000):
            batch = mnist.train.next_batch(50)
            # 100 Step마다 training 데이터셋에 대한 정확도를 출력한다.
            if i % 100 == 0:
                train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
                print('step %d, training accuracy %g' % (i, train_accuracy))
        # 50% 확률의 Dropout을 이용해서 학습을 진행한다.
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    # 테스트 데이터에 대한 정확도를 출력한다.
    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
